In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException, ElementClickInterceptedException
import undetected_chromedriver as uc
import json
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
link_produk = 'https://www.sankelux.co.id/search?keyword=inverter'
path = r"C:/Users/KBIBKU- BNBYTO/Documents/Scraping Data Komponen plts/chromedriver-win64/chromedriver.exe"

In [3]:
options = uc.ChromeOptions()
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

driver.set_window_size(1300,800)
driver.get(link_produk)
time.sleep(5)

In [4]:
halaman = 1
product_data = []
duplicate_count = 0
processed_products = set()  # Set untuk menyimpan ID produk yang sudah diproses

while True:
    print(f"Scraping halaman ke-{halaman}")
    produk_list = driver.find_elements(By.XPATH, "//div[contains(@class, 'col-md-3 col-sm-3 col-xs-6 no-padding-highlight item')]")
    print(f"Jumlah elemen ditemukan: {len(produk_list)}")
    
    # Reset counter duplikat untuk halaman baru
    page_duplicate_count = 0
    
    # Proses semua produk di halaman saat ini
    for i, produk in enumerate(produk_list, start=1):
        print(f"Proses data ke-{i}")
        try:
            # Tangkap teks dan atribut dengan find_elements
            nama = produk.find_elements(By.XPATH, ".//h5[contains(@class, 'product-highlight-title')]")
            harga = produk.find_elements(By.XPATH, ".//p[contains(@class, 'priceFix')]")
            link = produk.find_elements(By.XPATH, ".//a")
            
            # Ambil data jika ada, jika kosong beri nilai None
            nama_text = nama[0].text if nama else "None"
            harga_text = harga[0].text if harga else "None"
            link_href = link[0].get_attribute('href') if link else "None"
            
            # Buat unique identifier untuk produk ini
            product_id = f"{nama_text}_{harga_text}"
            
            # Periksa apakah produk ini sudah diproses sebelumnya
            if product_id in processed_products:
                print(f"Produk '{nama_text}' sudah diproses sebelumnya.")
                page_duplicate_count += 1
                if page_duplicate_count >= 3:
                    print(f"Ditemukan {page_duplicate_count} produk duplikat. Sepertinya kita sudah di halaman terakhir.")
                    break
                continue
                
            # Tambahkan ke set produk yang sudah diproses
            processed_products.add(product_id)
            
            # Cetak hasil
            print(f"Nama: {nama_text}")
            print(f"harga: {harga_text}")
            print(f"Link: {link_href}")
            print('-------')
            
            product_data.append({
                "nama produk": nama_text,
                "harga": harga_text,
                "link": link_href
            })  
        except Exception as e:
            print(f"Error memproses data ke-{i}: {e}")
    
    # Jika sudah ditemukan 3 produk duplikat, keluar dari loop utama
    if page_duplicate_count >= 3:
        print("Ditemukan 3 atau lebih produk duplikat. Scraping selesai.")
        break
    
    # Coba klik tombol next
    try:
        # Cari tombol next berdasarkan struktur HTML
        tombol_next = driver.find_element(By.CSS_SELECTOR, "a.next")
        
        # Klik pada tombol next
        tombol_next.click()
        print(f"Berhasil klik tombol halaman selanjutnya.")
        halaman += 1
        time.sleep(3)  # Tunggu halaman dimuat
            
    except Exception as e:
        print(f"Error saat klik tombol next: {e}")
        print("Sepertinya sudah mencapai halaman terakhir atau terjadi error. Scraping selesai.")
        break

print(f"Total produk yang berhasil di-scrape: {len(product_data)}")

Scraping halaman ke-1
Jumlah elemen ditemukan: 16
Proses data ke-1
Nama: Hybrid Power Back up UPS 1 Kva (Hypo 1K)
harga: Rp. 3.330.000
Link: https://www.sankelux.co.id/product?id=59
-------
Proses data ke-2
Nama: Hybrid Power Back up UPS 2 Kva (Hypo 2K)
harga: Rp. 5.499.450
Link: https://www.sankelux.co.id/product?id=60
-------
Proses data ke-3
Nama: Hybrid Power Back up UPS 3 Kva V series (Hypo V 3K)
harga: Rp. 5.499.450
Link: https://www.sankelux.co.id/product?id=28
-------
Proses data ke-4
Nama: Hybrid Power Back up UPS 5 Kva Paralel 9 unit
harga: Rp. 17.189.500
Link: https://www.sankelux.co.id/product?id=26
-------
Proses data ke-5
Nama: Hybrid Power Back up UPS V series 5 Kva (Hypo V 5K)
harga: Rp. 9.165.750
Link: https://www.sankelux.co.id/product?id=5
-------
Proses data ke-6
Nama: Hybrid Power Back-up UPS 3 Kva (Hypo 3K)
harga: Rp. 8.446.882
Link: https://www.sankelux.co.id/product?id=62
-------
Proses data ke-7
Nama: Inverter IFS 10 KW - 3 phase
harga: Rp. 37.631.250
Link: htt

In [5]:
df = pd.DataFrame(product_data)

In [6]:
df

,nama produk,harga,link
0,Hybrid Power Back up UPS 1 Kva (Hypo 1K),Rp. 3.330.000,https://www.sankelux.co.id/product?id=59
1,Hybrid Power Back up UPS 2 Kva (Hypo 2K),Rp. 5.499.450,https://www.sankelux.co.id/product?id=60
2,Hybrid Power Back up UPS 3 Kva V series (Hypo ...,Rp. 5.499.450,https://www.sankelux.co.id/product?id=28
3,Hybrid Power Back up UPS 5 Kva Paralel 9 unit,Rp. 17.189.500,https://www.sankelux.co.id/product?id=26
4,Hybrid Power Back up UPS V series 5 Kva (Hypo ...,Rp. 9.165.750,https://www.sankelux.co.id/product?id=5
5,Hybrid Power Back-up UPS 3 Kva (Hypo 3K),Rp. 8.446.882,https://www.sankelux.co.id/product?id=62
6,Inverter IFS 10 KW - 3 phase,Rp. 37.631.250,https://www.sankelux.co.id/product?id=33
7,Inverter IFS V - 3 KW,Rp. 8.548.320,https://www.sankelux.co.id/product?id=6
8,Inverter IFS V - 5 KW,Rp. 13.476.480,https://www.sankelux.co.id/product?id=37
9,Inverter IFSP 5 KW 230V,Rp. 23.067.615,https://www.sankelux.co.id/product?id=72


In [7]:
df.duplicated().sum()

0

In [19]:
df['no'] = df.index + 1

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nama produk  38 non-null     object
 1   harga        38 non-null     object
 2   link         38 non-null     object
dtypes: object(3)
memory usage: 1.0+ KB


In [15]:
df

,nama produk,harga,link
0,Hybrid Power Back up UPS 1 Kva (Hypo 1K),Rp. 3.330.000,https://www.sankelux.co.id/product?id=59
1,Hybrid Power Back up UPS 2 Kva (Hypo 2K),Rp. 5.499.450,https://www.sankelux.co.id/product?id=60
2,Hybrid Power Back up UPS 3 Kva V series (Hypo ...,Rp. 5.499.450,https://www.sankelux.co.id/product?id=28
3,Hybrid Power Back up UPS 5 Kva Paralel 9 unit,Rp. 17.189.500,https://www.sankelux.co.id/product?id=26
4,Hybrid Power Back up UPS V series 5 Kva (Hypo ...,Rp. 9.165.750,https://www.sankelux.co.id/product?id=5
5,Hybrid Power Back-up UPS 3 Kva (Hypo 3K),Rp. 8.446.882,https://www.sankelux.co.id/product?id=62
6,Inverter IFS 10 KW - 3 phase,Rp. 37.631.250,https://www.sankelux.co.id/product?id=33
7,Inverter IFS V - 3 KW,Rp. 8.548.320,https://www.sankelux.co.id/product?id=6
8,Inverter IFS V - 5 KW,Rp. 13.476.480,https://www.sankelux.co.id/product?id=37
9,Inverter IFSP 5 KW 230V,Rp. 23.067.615,https://www.sankelux.co.id/product?id=72


## Data deskripsi produk

In [9]:
deskripsi_data = [] 
for idx, url in enumerate(df['link']):
    print(f"\nMemulai scraping untuk produk {idx + 1}: {url}")
    driver.get(url)
    # Tunggu halaman dimuat sepenuhnya
    time.sleep(5)
    
    try:
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC
        
        deskripsi = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'tab-pane fade active in')]"))
        )
        
        # Ambil teks dari elemen yang ditemukan
        deskripsi_text = deskripsi.text
        
        # Cetak hasil
#         print(f"deskripsi: {deskripsi_text}")
#         print('-------')
        deskripsi_data.append({
            "no": idx+1,
            "spesifikasi": deskripsi_text
        })
    except Exception as e:
        print(f"Error saat scraping produk {idx + 1}: {e}")
        deskripsi_data.append({"spesifikasi": "None"})


Memulai scraping untuk produk 1: https://www.sankelux.co.id/product?id=59

Memulai scraping untuk produk 2: https://www.sankelux.co.id/product?id=60

Memulai scraping untuk produk 3: https://www.sankelux.co.id/product?id=28

Memulai scraping untuk produk 4: https://www.sankelux.co.id/product?id=26

Memulai scraping untuk produk 5: https://www.sankelux.co.id/product?id=5

Memulai scraping untuk produk 6: https://www.sankelux.co.id/product?id=62

Memulai scraping untuk produk 7: https://www.sankelux.co.id/product?id=33

Memulai scraping untuk produk 8: https://www.sankelux.co.id/product?id=6

Memulai scraping untuk produk 9: https://www.sankelux.co.id/product?id=37

Memulai scraping untuk produk 10: https://www.sankelux.co.id/product?id=72

Memulai scraping untuk produk 11: https://www.sankelux.co.id/product?id=73

Memulai scraping untuk produk 12: https://www.sankelux.co.id/product?id=81

Memulai scraping untuk produk 13: https://www.sankelux.co.id/product?id=83

Memulai scraping untuk

In [10]:
spesifikasi = pd.DataFrame(deskripsi_data)

In [11]:
spesifikasi

,no,spesifikasi
0,1,INVERTER SANKELUX HYPO UPS 1 Kva ( 800W )\nRan...
1,2,INVERTER SANKELUX HYPO UPS 2 KVA ( 1.600 W )\n...
2,3,DESKRIPSI\n\nSankelux Hypo UPS V seriesÂ adala...
3,4,INVERTER SANKELUX HYPO UPS 5000 Watt\nRangkaia...
4,5,DESKRIPSI\nSankelux Hypo UPS V seriesÂ adalah ...
5,6,INVERTER SANKELUX HYPO UPS 3 KVA ( 2400 W )\nR...
6,7,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...
7,8,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...
8,9,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...
9,10,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...


In [22]:
spesifikasi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   no           38 non-null     int64 
 1   spesifikasi  38 non-null     object
dtypes: int64(1), object(1)
memory usage: 736.0+ bytes


In [12]:
spesifikasi['spesifikasi'].values[3]

'INVERTER SANKELUX HYPO UPS 5000 Watt\nRangkaian elektronika daya yang di gunakan untuk mengkonversikan tegangan searah (DC) kesuatu tegangan bolak balik (AC)\n\nDESKRIPSI\nRated PowerÂ  Â  Â  Â  Â  Â  Â  5000 VA/ 5000 Watt\nParalel CapabilityÂ  Â  Â  Â 9 units\nVoltageÂ  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â 230 VAC\nSelectable Voltage RangeÂ  Â  170-280 VAC (For Persona90-280 VAC ( For Home Appliance)\nFrequency Range Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â 50Hz / 60Hz ( auto sensing )\nAc Voltage Regule on Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â 230 VAC Â±5%\nSurge Power Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â 10000 KVA\nEsficiency (Peak) Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â  Â 90 %\n

In [19]:
import re

def bersihkan_spesifikasi(teks):
    if pd.isnull(teks):
        return teks
    # Ganti satu atau lebih karakter Â dan spasi/tab di belakangnya dengan ' '
    teks_bersih = re.sub(r'Â[\sÂ]*', ' ', teks).strip()
    return teks_bersih

In [20]:
spesifikasi['spesifikasi_formatted'] = spesifikasi['spesifikasi'].apply(bersihkan_spesifikasi)

In [21]:
spesifikasi['spesifikasi_formatted'].values[3]

'INVERTER SANKELUX HYPO UPS 5000 Watt\nRangkaian elektronika daya yang di gunakan untuk mengkonversikan tegangan searah (DC) kesuatu tegangan bolak balik (AC)\n\nDESKRIPSI\nRated Power 5000 VA/ 5000 Watt\nParalel Capability 9 units\nVoltage 230 VAC\nSelectable Voltage Range 170-280 VAC (For Persona90-280 VAC ( For Home Appliance)\nFrequency Range  50Hz / 60Hz ( auto sensing )\nAc Voltage Regule on  230 VAC  ±5%\nSurge Power  10000 KVA\nEsficiency (Peak)  90 %\nTransfer Time  10 ms ( for personal computers )\n 20 ms ( for home appliances )\nWaveform  Pure sine wave\nBatery Voltage  48 VDC\nFloating Charge Voltage  54 VDC\nOvercharge Protection  60 VDC\nMaximum Charge Current  20 A or 30 A\nMaximum pv array Power 4000 W\nMPPT Range@ Operating Voltage  60 VDC ~ 115 VDC\nMaximum PV Array Open cicuit Voltage  145 VDC\nMaximum Charging Current  60 A\nMaximum Eficiency  98%\nStandby Power Consumption  2 W  Dimension, D x W x H (mm)  120 x 295 x 468\nNet Weight  13.5\nHumidity 5% to 95%\nRelat

In [22]:
spesifikasi['spesifikasi_formatted'].values[2]

'DESKRIPSI\n\nSankelux Hypo UPS V series adalah Alat yang digunakan sebagai tenaga cadangan (Power Back-up) saat Listrik padam dengan menggunakan Tenaga Baterai (Accu). Alat ini di desain agar dapat bekerja secara otomatis saat listrik PLN padam/nyala kembali. Disamping bersifat Ramah lingkungan, Hypo UPS juga sangat aman digunakan karena telah dilengkapi pengaman. Umumnya Alat ini digunakan sebagai pelengkap ataupun bisa sebagai pengganti Genset. Sankelux Hypo UPS V series memiliki desain yang elegan sehingga tidak mengganggu estetika rumah maupun kantor Anda. Alat ini cocok digunakan di bidang usaha seperti Oil & Gas company, Resto, Salon, Tambak Udang serta bidang usaha lainnya yang sangat menjaga pasokan listrik tidak sampai terputus. Tersedia beberapa pilihan kapasitas daya dan dapat disesuaikan dengan kebutuhan.\n\nKEUNGGULAN\n\nHybrid Power Source\nDapat memilih menggunakan Sumber daya dari PLN, Accu maupun Tenaga Surya.\nBuilt-in Solar Charge Controller Dilengkapi Solar Charge 

In [24]:
spesifikasi

,no,spesifikasi,spesifikasi_formatted
0,1,INVERTER SANKELUX HYPO UPS 1 Kva ( 800W )\nRan...,INVERTER SANKELUX HYPO UPS 1 Kva ( 800W )\nRan...
1,2,INVERTER SANKELUX HYPO UPS 2 KVA ( 1.600 W )\n...,INVERTER SANKELUX HYPO UPS 2 KVA ( 1.600 W )\n...
2,3,DESKRIPSI\n\nSankelux Hypo UPS V seriesÂ adala...,DESKRIPSI\n\nSankelux Hypo UPS V series adalah...
3,4,INVERTER SANKELUX HYPO UPS 5000 Watt\nRangkaia...,INVERTER SANKELUX HYPO UPS 5000 Watt\nRangkaia...
4,5,DESKRIPSI\nSankelux Hypo UPS V seriesÂ adalah ...,DESKRIPSI\nSankelux Hypo UPS V series adalah A...
5,6,INVERTER SANKELUX HYPO UPS 3 KVA ( 2400 W )\nR...,INVERTER SANKELUX HYPO UPS 3 KVA ( 2400 W )\nR...
6,7,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...,Sankelux IFS adalah Inverter Hybrid yang dapat...
7,8,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...,Sankelux IFS adalah Inverter Hybrid yang dapat...
8,9,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...,Sankelux IFS adalah Inverter Hybrid yang dapat...
9,10,Sankelux IFSÂ adalah Inverter Hybrid yang dapa...,Sankelux IFS adalah Inverter Hybrid yang dapat...


In [25]:
df['spesifikasi'] = spesifikasi['spesifikasi_formatted']

In [27]:
df = df[['nama produk','harga','spesifikasi', 'link']]

In [28]:
df

,nama produk,harga,spesifikasi,link
0,Hybrid Power Back up UPS 1 Kva (Hypo 1K),Rp. 3.330.000,INVERTER SANKELUX HYPO UPS 1 Kva ( 800W )\nRan...,https://www.sankelux.co.id/product?id=59
1,Hybrid Power Back up UPS 2 Kva (Hypo 2K),Rp. 5.499.450,INVERTER SANKELUX HYPO UPS 2 KVA ( 1.600 W )\n...,https://www.sankelux.co.id/product?id=60
2,Hybrid Power Back up UPS 3 Kva V series (Hypo ...,Rp. 5.499.450,DESKRIPSI\n\nSankelux Hypo UPS V series adalah...,https://www.sankelux.co.id/product?id=28
3,Hybrid Power Back up UPS 5 Kva Paralel 9 unit,Rp. 17.189.500,INVERTER SANKELUX HYPO UPS 5000 Watt\nRangkaia...,https://www.sankelux.co.id/product?id=26
4,Hybrid Power Back up UPS V series 5 Kva (Hypo ...,Rp. 9.165.750,DESKRIPSI\nSankelux Hypo UPS V series adalah A...,https://www.sankelux.co.id/product?id=5
5,Hybrid Power Back-up UPS 3 Kva (Hypo 3K),Rp. 8.446.882,INVERTER SANKELUX HYPO UPS 3 KVA ( 2400 W )\nR...,https://www.sankelux.co.id/product?id=62
6,Inverter IFS 10 KW - 3 phase,Rp. 37.631.250,Sankelux IFS adalah Inverter Hybrid yang dapat...,https://www.sankelux.co.id/product?id=33
7,Inverter IFS V - 3 KW,Rp. 8.548.320,Sankelux IFS adalah Inverter Hybrid yang dapat...,https://www.sankelux.co.id/product?id=6
8,Inverter IFS V - 5 KW,Rp. 13.476.480,Sankelux IFS adalah Inverter Hybrid yang dapat...,https://www.sankelux.co.id/product?id=37
9,Inverter IFSP 5 KW 230V,Rp. 23.067.615,Sankelux IFS adalah Inverter Hybrid yang dapat...,https://www.sankelux.co.id/product?id=72


In [29]:
df.to_excel('inverter_products_from_sankelux.xlsx', index=False)